In [ ]:
import matplotlib.pyplot as plt
import time

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import numpy as np
from itertools import combinations

In [ ]:
from pkg.tf_transform import *
from pkg.tf_robot import *
from pkg.constraint import *
from pkg.info import *
from pkg.tf_utils import *
from pkg.rotation_utils import *
from pkg.utils import *
from pkg.ur10 import *
from pkg.geometry import *
from pkg.collision import *

# Load urdf 

In [ ]:
from urdf_parser_py.urdf import URDF
from pkg.ur10 import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE
from pkg.joint_utils import *
urdf_content = URDF.from_xml_file(URDF_PATH)

In [ ]:
link_names = LINK_NAMES
base_name = LINK_NAMES[0]
joint_names = JOINT_NAMES
link_info_list = get_link_info_list(link_names, urdf_content)

# Visualize

In [ ]:
from pkg.ros_rviz import *
pub, joints, rate = get_publisher(JOINT_NAMES)

gitem_list, gframe_dict= get_link_items_offsets(color=(0,1,0,0.5), display=True)
gitem_list += [
    GeometryItem(name='box1', gtype=GeoType.BOX, dims=[0.1,0.1,0.1], color=(0,1,0,1), display=True, collision=True)
]
gframe_dict.update({"box1":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.5,0,0)), "world")
                   })
marker_list = set_markers(gitem_list, gframe_dict, urdf_content)
show_motion([np.array([0]*6)], marker_list, 
            [[gframe_dict[gitem.name] for gitem in gitem_list]], 
            pub, joints, error_skip=1e-6, period=1e-6)

# Constraint

In [ ]:
def get_adjacent_links(link_name, urdf_content):
    adjacent_links = [link_name]
    for k, v in urdf_content.joint_map.items():
        if v.parent == link_name:
            adjacent_links += [v.child]
        if v.child == link_name:
            adjacent_links += [v.parent]
    return list(set(adjacent_links))

In [ ]:
class ObjectLayer(layers.Layer):
    def __init__(self, gitem, N_sim, *args, **kwargs):
        self.gitem, self.N_sim = gitem, N_sim
        super(ObjectLayer, self).__init__(*args, **kwargs)
        
    def set_frame(self, Toff_list, link_idx_list, N_link):
        self.Toff_list = tf.constant(Toff_list) # (N_sim, 4,4)
        self.link_one_hot = tf.reshape(tf.one_hot(link_idx_list, N_link), (N_sim,N_link,1,1)) # (N_sim, N_link)
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass
    
    @tf.function
    def get_vertice(self):
        return self.vertice

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, input=None):
        T_all = input # (N_sim, N_link, 4,4)
        T_act = K.sum(T_all*self.link_one_hot, axis=1) # (N_sim, 4,4)
        T_bo = tf.matmul(T_act, self.Toff_list) # (N_sim, 4,4)
        return T_bo

# Graph

In [ ]:
class GraphModel(tf.keras.Model):
    def __init__(self, robot_info, gitem_list, urdf_content, N_sim, N_col,
                 alpha_jc=5, alpha_fc=200, alpha_jl=1, alpha_cl=1,alpha_cs=0, 
                 LIM=np.pi, LIM_BOUND=1e-1, COL_BOUND=1e-1, learning_rate=5e-3):
        super(GraphModel, self).__init__()
        self.alpha_jc = alpha_jc
        self.alpha_fc = alpha_fc
        self.alpha_jl = alpha_jl
        self.alpha_cl = alpha_cl
        self.alpha_cs = alpha_cs
        self.N_sim, self.N_col = N_sim, N_col
        self.LIM = LIM
        self.LIM_BOUND = LIM_BOUND
        self.COL_BOUND = COL_BOUND
        self.learning_rate = learning_rate
        self.robot_info = robot_info
        self.robot = RobotLayer(
            robot_info.link_info_list, rname = robot_info.rname, dim=N_sim)
        self.joint_constraint = JointConstraintLoss(self.robot)
        self.frame_constraint = FrameConstraintLoss()
        self.robot_base = [robot_info.base_frame]*N_sim
        self.set_custom_loss(lambda *args: tf.constant(0.0), 0)
        self.link_name_list = self.robot.link_name_list
        self.link_idx_dict = {name: idx for name, idx in zip(self.link_name_list, range(len(self.link_name_list)))}
        self.adjacency_list = [
            list(map(
                lambda x: self.link_idx_dict[x],
                [lname for lname in get_adjacent_links(link_name, urdf_content) if lname in self.link_name_list]
            )) for link_name in self.link_name_list
        ]
        link_num = len(self.adjacency_list)
        self.adjacency_mat = np.zeros((link_num,link_num), dtype=np.bool)
        for i_adj, adj in zip(range(link_num), self.adjacency_list):
            self.adjacency_mat[i_adj, adj] = True
            
        self.object_dict = {}
        self.object_name_list = []
        self.object_vertice_list = []
        self.object_link_idx_list = []
        self.object_collision_flags = []
        for gitem in gitem_list:
            self.object_dict[gitem.name] = ObjectLayer(gitem, N_sim)
            self.object_collision_flags += [gitem.collision]
            self.object_name_list += [gitem.name]
            self.object_vertice_list += [np.pad(gitem.get_vertice(),((0,0),(0,1)),'constant', constant_values=(1))]
            self.object_link_idx_list += [0]
        self.object_name_list = self.object_name_list
        self.object_vertice_mat = tf.constant(self.object_vertice_list, dtype="float32")
        self.build_collision_combinatiosn()
        self.optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
        self.flag_default = get_flag_default(N_sim, N_col)
        self.dist_default = get_dist_default(N_sim, N_col)
        self.x_batch, self.y_batch = get_xy_batch(N_sim, N_col)
        
    def build_collision_combinatiosn(self):
        obj_idx_list = np.arange(len(self.object_name_list))
        self.col_combinations = list(combinations(obj_idx_list,2))
        for i_c in range(len(self.col_combinations)):
            i_c_back = len(self.col_combinations)-1-i_c
            cmb = self.col_combinations[i_c_back]
            if not(self.object_collision_flags[cmb[0]] 
                   and self.object_collision_flags[cmb[1]]):
                del self.col_combinations[i_c_back]
        self.col_combinations = np.array(self.col_combinations)
        self.len_combs = len(self.col_combinations)
            
    def assign_frame_dict(self, gframeset_list):
        frame_dict = {k: [] for k in self.object_dict.keys()}
        link_dict = {k: [] for k in self.object_dict.keys()}
        for gframeset in gframeset_list:
            for k, gframe in gframeset.items():
                frame_dict[k] += [gframe.Toff]
                link_dict[k] += [self.robot.link_name_list.index(gframe.link_name)]
        for k in frame_dict.keys():
            self.object_dict[k].set_frame(np.array(frame_dict[k]), np.array(link_dict[k]), self.robot.len_Q)
            i_obj = self.get_object_index(k)
            self.object_link_idx_list[i_obj] = np.array(link_dict[k])
        self.object_link_idx_mat = np.transpose(self.object_link_idx_list)
            
    def get_object_index(self, name):
        return graph.object_name_list.index(name)
            
    @tf.function
    def assign_Q(self, Q):
        self.robot.assign_Q(Q)
            
    @tf.function
    def get_Q(self):
        return self.robot.get_Q()

    @tf.function
    def call(self, inputs=None):
        T_all = self.robot(self.robot_base)
        Tbo_all = []
        for obj_name in self.object_name_list:
            Tbo_all += [self.object_dict[obj_name](T_all)] #(Nobj,N_sim,4,4)
        Tbo_all = K.stack(Tbo_all, axis=1) #(N_sim,Nobj,4,4)
        return T_all, Tbo_all
    
    @tf.function
    def calc_joint_limit(self):
        Q = self.get_Q()
        return K.sum(1/((K.min(((self.LIM-Q), (self.LIM+Q)), axis=-1)/self.LIM_BOUND)**3))
    
    @tf.function
    def calc_loss(self, T_all, Tbo_all, Qtar, binQ, Ttar, binT):
        jl_loss = self.calc_joint_limit()
        jc_loss = self.joint_constraint((Qtar, binQ))
        fc_loss = self.frame_constraint((T_all[:,-1,:,:],Ttar, binT))
#         cl_loss = self.calc_collision_loss(T_all, Tbo_all)
        return self.alpha_jc*jc_loss + self.alpha_fc*fc_loss + \
                self.alpha_jl*jl_loss + self.alpha_cs*self.calc_custom_loss(T_all, Tbo_all) #
#                 + self.alpha_cl*cl_loss 
                
    
    @tf.function
    def forward(self, Qtar, binQ, Ttar, binT):
        T_all, Tbo_all= graph(None)
        loss = self.calc_loss(T_all, Tbo_all, Qtar, binQ, Ttar, binT)
        return loss
    
    @tf.function
    def update_once(self, Qtar, binQ, Ttar, binT, max_gradient):
        with tf.GradientTape() as g:
            # Forward pass.
            loss = self.forward(Qtar, binQ, Ttar, binT)

        # Variables to update, i.e. trainable variables.
        trainable_variables = self.trainable_variables

        # Compute gradients.
        gradients = g.gradient(loss, trainable_variables)
        gradients = tf.unstack(clip_gradient(gradients, max_gradient))
    
        # Update W and b following gradients.
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss
    
    def set_custom_loss(self, custom_loss, alpha_cs, *args, **kwargs):
        self.custom_loss = custom_loss
        self.alpha_cs = alpha_cs
        self.cl_args = args
        self.cl_kwargs = kwargs
        
    def calc_custom_loss(self, T_all, Tbo_all):
        return self.custom_loss(self, T_all, Tbo_all, *self.cl_args, **self.cl_kwargs)

    @tf.function
    def collision_condition(self, col_comb):
        link_comb = tf.gather(self.object_link_idx_mat, col_comb, axis=-1)
        adjacency_masked = tf.logical_not(tf.gather_nd(self.adjacency_mat, link_comb))
        return adjacency_masked
    @tf.function
    def _combination_cond(self, col_combs, col_mask, iter):
        return True
    @tf.function
    def _combination_loop(self, col_combs, col_mask, iter):
        col_comb = tf.gather(col_combs, iter, axis=0)
        col_mask = tf.logical_or(col_mask, 
                                 tf.logical_and(tf.cast(tf.one_hot(iter, self.len_combs, axis=0), dtype=tf.bool),
                                                tf.expand_dims(self.collision_condition(col_comb), axis=1)
                                               )
                                )
        return col_combs, col_mask, iter+1

    @tf.function
    def test_collision(self, T_all, Tbo_all):
        Pbo_all = tf.gather(tf.gather(Tbo_all, [0,1,2], axis=-2), 3, axis=-1)

        col_combs = self.col_combinations
        col_mask = tf.zeros((self.N_sim, self.len_combs),tf.bool)
        col_combs, col_mask, iter = tf.while_loop(cond=self._combination_cond, body=self._combination_loop, 
                                                         loop_vars=(col_combs, col_mask, K.constant(0, tf.int64)), 
                                                         parallel_iterations=10,maximum_iterations=self.len_combs)
        col_combs_masked_batch = tf.boolean_mask(
                tf.tile(tf.expand_dims(col_combs,axis=0), [self.N_sim, 1, 1]), 
                col_mask, 
                axis=0)
        batch_flat = tf.repeat(np.reshape(np.arange(self.N_sim), (self.N_sim, 1)), (self.N_col,)*self.N_sim,axis=0)
        col_combs_masked_flat = tf.concat([batch_flat, col_combs_masked_batch], axis=-1)
        vtx_tf = tf.gather(
            K.sum(tf.expand_dims(Tbo_all, axis=-3)*tf.expand_dims(self.object_vertice_mat, axis=-2), axis=-1), 
            [0,1,2], axis=-1)
        vtx_tf_1 = tf.reshape(tf.gather_nd(vtx_tf, tf.gather(col_combs_masked_flat, [0,1], axis=1)), (self.N_sim, self.N_col, -1,3))
        vtx_tf_2 = tf.reshape(tf.gather_nd(vtx_tf, tf.gather(col_combs_masked_flat, [0,2], axis=1)), (self.N_sim, self.N_col, -1,3))
        Pbo_all_1 = tf.reshape(tf.gather_nd(Pbo_all, tf.gather(col_combs_masked_flat, [0,1], axis=1)), (self.N_sim, self.N_col,3))
        Pbo_all_2 = tf.reshape(tf.gather_nd(Pbo_all, tf.gather(col_combs_masked_flat, [0,2], axis=1)), (self.N_sim, self.N_col,3))
        v_batch = tf.expand_dims(Pbo_all_1-Pbo_all_2, axis=-2)
        v_batch = tf.linalg.cross(v_batch, self.x_batch)+tf.linalg.cross(v_batch, self.y_batch)
        dist, flag = test_collision_batch(vtx_tf_1, vtx_tf_2, 
                             v_batch, self.flag_default, self.dist_default)
        return dist, flag, col_combs_masked_flat 
    
    @tf.function
    def calc_collision_loss(self, T_all, Tbo_all):
        dist, flag, _  = self.test_collision(T_all, Tbo_all)
        return K.min(1/(dist/self.COL_BOUND)**3)

In [ ]:
N_sim = 50
N_col = 21
N_joints = 9
DOF = 6
robot_info = RobotInfo(link_info_list, rname = "rbt1", base_frame=np.identity(4,dtype=np.float32))
graph = GraphModel(robot_info=robot_info, gitem_list=gitem_list, urdf_content=urdf_content, 
                   N_sim=N_sim, N_col=N_col, learning_rate=5e-3, 
                   alpha_cl=1)
gtimer = GlobalTimer()

In [ ]:
Q_ = np.array([(0, 0,)+tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/3)+(0,) for _ in range(N_sim)], dtype=np.float32)
graph.assign_Q(Q_)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_frame_dict(gframe_dict_list)

# SGD Optimizer

In [ ]:
Q_ = np.array([(0, 0,)+tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100)+(0,) for _ in range(N_sim)], dtype=np.float32)
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,-0.05)).astype(np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)

In [ ]:
time_vec = []
Q_list = [list(map(lambda x: x.numpy(), graph.get_Q()))]
# Tbo_all_list = [np.array([[gframe_dict[gitem.name].Toff for gitem in gitem_list]]*N_sim)]
N_iter = 100
# Run training for the given number of steps.
Qtar, binQ, Ttar, binT = (np.zeros((N_sim, N_joints), dtype='float32'), np.zeros(N_sim, dtype='float32'),
                      np.array([Ttar_]*N_sim), np.ones(N_sim, dtype='float32'))
dist_list = []
loss_list = []
jl_loss_list = []
jc_loss_list = []
fc_loss_list = []
cl_loss_list = []
gradients_list = []
gtimer.reset()
for _ in range(N_iter):
    # Run the optimization to update W and b values.
    T_all, Tbo_all = graph(None)
    dist_list += [K.min(graph.test_collision(T_all, Tbo_all)[0]).numpy()]
    jl_loss = graph.calc_joint_limit()
    jc_loss = graph.joint_constraint((Qtar, binQ))
    fc_loss = graph.frame_constraint((T_all[:,-1,:,:],Ttar, binT))
    cl_loss = graph.calc_collision_loss(T_all, Tbo_all)
    gtimer.tic("update")
    loss = graph.update_once(Qtar, binQ, Ttar, binT, max_gradient=10)
#     max_gradient=10

#     with tf.GradientTape() as g:
#         # Forward pass.
#         loss = graph.forward(Qtar, binQ, Ttar, binT)

#     # Variables to update, i.e. trainable variables.
#     trainable_variables = graph.trainable_variables

#     # Compute gradients.
#     gradients = g.gradient(loss, trainable_variables)
#     gradients = tf.unstack(clip_gradient(gradients, max_gradient))

#     # Update W and b following gradients.
#     graph.optimizer.apply_gradients(zip(gradients, graph.trainable_variables))
    gtimer.toc("update")
    gtimer.tic("record")
    Q_list += [graph.get_Q().numpy()]
    loss_list += [loss]
#     gradients_list += [gradients]
    jl_loss_list += [jl_loss]
    jc_loss_list += [jc_loss]
    fc_loss_list += [fc_loss]
    cl_loss_list += [cl_loss]
    gtimer.toc("record")
gtimer.print_time_log()
print("loss: {}".format(loss))

In [ ]:
plt.plot(loss_list, label="total")

In [ ]:
plt.plot(loss_list, label="total")
plt.plot(np.multiply(jl_loss_list, graph.alpha_jl), label="joint limit")
plt.plot(np.multiply(jc_loss_list, graph.alpha_jc), label="joint target")
plt.plot(np.multiply(fc_loss_list, graph.alpha_fc), label="frame target")
plt.plot(np.multiply(cl_loss_list, graph.alpha_cl), label="collision dist")
plt.legend()
axes = plt.gca()
# axes.set_ylim([-1e1,5e1])
# axes.set_xlim([0,1e1])

In [ ]:
for i_sim in range(1): # N_sim):
    for i_iter in range(1,N_iter+1):
        q = Q_list[i_iter][i_sim]
        gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
        pose_list = [q[2:-1]]
        gframevec_list = [gframevec]
        show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)
        time.sleep(10e-2)

In [ ]:
Q_all = np.array(Q_list)
Q_all_i = np.array([Q_all[i_iter, i_sim,:] for i_iter in range(N_iter)])
plt.plot(Q_all_i)

In [ ]:
plt.plot(Q_all_i[1:,:]-Q_all_i[:-1,:])

# Collision test

In [ ]:
Q_ = np.array([(0, 0,)+tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100)+(0,) for _ in range(N_sim)], dtype=np.float32)
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)

In [ ]:
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,-0.05)).astype(np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
T_all, Tbo_all = graph(None)
show_motion([graph.get_Q().numpy()[0][2:-1]], marker_list, [[gframe_dict[gitem.name] for gitem in gitem_list]], pub, joints, error_skip=1e-6, period=1e-6)

In [ ]:
T_all, Tbo_all = graph(None)
gtimer.reset()
for _ in range(10):
    gtimer.tic("ctest")
    dist, flag, mask = graph.test_collision(T_all, Tbo_all)
    gtimer.toc("ctest")
print(gtimer)
# dist[0]
# gtimer.reset()
# for _ in range(100):
#     gtimer.tic("col")
#     dist, flag = collision_loss_test(graph, T_all, Tbo_all, flag_default, dist_default)
#     gtimer.toc("col")
# gtimer.print_time_log()

In [ ]:
graph.calc_collision_loss(T_all, Tbo_all)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot([0, N_col],[0,0],'-ob')
plt.grid()
dist0 = dist[0]
mask0 = mask[:N_col]
for i_m in range(len(mask0)):
    plt.plot([i_m, i_m],mask0[i_m,1:],'-o')
plt.plot(dist0*10,'-ok',linewidth=2)

In [ ]:
graph.object_name_list

# Collision error case 

In [ ]:
Q_error = np.loadtxt("Q_error.csv",delimiter=",",dtype=np.float32)
graph.assign_Q(Q_error)
graph.assign_frame_dict(gframe_dict_list)

T_all, Tbo_all = graph(None)
dist = K.min(graph.test_collision(T_all, Tbo_all)[0]).numpy()
cl_loss = graph.calc_collision_loss(T_all, Tbo_all)
print("dist: {}".format(dist))

In [ ]:
q = Q_error[0]
gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
pose_list = [q[2:-1]]
gframevec_list = [gframevec]
show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import mpl_toolkits.mplot3d as mplot3d

In [ ]:
fig = plt.figure()
sub = fig.add_subplot(1,1,1,projection="3d")
sub.plot(x,y,z)
plt.show()

In [ ]:
t = np.linspace(0,4*np.pi,500)
x = np.sin(t)
x = np.cos(t)
y = np.sin(t)
z = t